In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('safe_train.csv')
df = df.drop(['Unnamed: 0'], axis = 1)

In [3]:
print("Dataset has {} entries and {} features".format(*df.shape))

Dataset has 1192 entries and 28 features


In [4]:
df

,country,isonum,isocode,year,rank_safe,safe,shelter,shelter_year,food,food_year,...,tdeathslnl5,tdeathslnl5_year,refugee_oriln,refugee_oriln_year,bdeathslnl,bdeathslnl_year,rdeathslnl,rdeathslnl_year,homilnl,homilnl_year
0,Afghanistan,4,AFG,2007,149,37.240280,***,2008,***,2008,...,6.059574,2007,11.679785,2007,5.564811,2007,2.265712,--,1.392297,2009
1,Angola,24,AGO,2007,137,53.325123,***,2011,***,2011,...,-0.693147,2007,9.180264,2007,0.264804,2007,2.977611,--,2.383776,2012
2,Albania,8,ALB,2007,61,68.989883,***,2009,***,2009,...,-0.693147,2007,8.549651,2007,-4.298536,2007,2.186051,2007,1.198836,2007
3,United Arab Emirates,784,ARE,2007,16,80.370224,***,2009,***,2009,...,-0.693147,2007,3.936658,2007,-4.298536,2007,2.147497,--,-0.231057,2010
4,Argentina,32,ARG,2007,65,68.519714,***,2007,***,2007,...,-0.693147,2007,3.377487,2007,-4.298536,2007,2.379546,2007,1.958831,2007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1187,Vietnam,704,VNM,2014,58,69.328110,***,2014,***,2014,...,-0.693147,2014,8.160349,2013,-4.298536,2014,2.511276,--,0.413376,2011
1188,Yemen,887,YEM,2014,135,53.523056,***,2014,***,2014,...,6.516417,2014,4.554843,2013,-4.298536,2014,2.692122,--,1.945797,2013
1189,South Africa,710,ZAF,2014,124,56.250931,***,2014,***,2014,...,0.622703,2014,2.074113,2013,-4.298536,2014,2.397895,2013,3.461466,2013
1190,Zambia,894,ZMB,2014,116,57.675217,***,2014,***,2014,...,-0.693147,2014,2.722414,2013,-4.298536,2014,2.635879,--,1.817873,2010


In [5]:
y = df['safe']
df = df.drop(['rank_safe', 'safe'], axis = 1)

In [6]:
from featuretools.selection import remove_low_information_features, remove_highly_null_features, remove_single_value_features, remove_highly_correlated_features

df = remove_low_information_features(df)

df = remove_highly_null_features(df)

df = remove_single_value_features(df)

df = remove_highly_correlated_features(df)

X = df

In [7]:
import evalml
import woodwork as ww

problem_type = 'regression'
objective =  'auto'

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)


automl = evalml.automl.AutoMLSearch(X_train, y_train, problem_type=problem_type, objective = objective)
automl.search()

best_pipeline = automl.best_pipeline

Using default limit of max_batches=1.

Generating pipelines to search over...
*****************************
* Beginning pipeline search *
*****************************

Optimizing for R2. 
Greater score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: catboost, random_forest, extra_trees, linear_model, xgboost, decision_tree, lightgbm



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Batch 1: (1/9) Mean Baseline Regression Pipeline        Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean R2: -0.005
High coefficient of variation (cv >= 0.2) within cross validation scores. Mean Baseline Regression Pipeline may not perform as estimated on unseen data.
Batch 1: (2/9) Decision Tree Regressor w/ Imputer + ... Elapsed:00:00
	Starting cross validation
	Finished cross validation - mean R2: 0.900
Batch 1: (3/9) Extra Trees Regressor w/ Imputer + On... Elapsed:00:01
	Starting cross validation
	Finished cross validation - mean R2: 0.930
Batch 1: (4/9) XGBoost Regressor w/ Imputer + One Ho... Elapsed:00:03
	Starting cross validation
	Finished cross validation - mean R2: 0.958
Batch 1: (5/9) CatBoost Regressor w/ Imputer            Elapsed:00:38
	Starting cross validation
	Finished cross validation - mean R2: 0.333
Batch 1: (6/9) Random Forest Regressor w/ Imputer + ... Elapsed:00:41
	Starting cross validation
	Finished cross validation - mean R2: 0.931


In [8]:
best_pipeline.graph()

RuntimeError: To graph entity sets, a graphviz backend is required.
Install the backend using one of the following commands:
  Mac OS: brew install graphviz
  Linux (Ubuntu): sudo apt-get install graphviz
  Windows: conda install python-graphviz


In [9]:
best_pipeline.feature_importance

,feature,importance
0,warcasual,0.401623
1,homilnl,0.201393
2,pts,0.122676
3,bdeathslnl,0.117034
4,rdeathslnl_year_--,0.030746
...,...,...
72,stolen_year_2014,0.000000
73,warcasual_year_2013,0.000000
74,pts_year_--,0.000000
75,pts_year_2010,0.000000


In [10]:
predictions = best_pipeline.predict(X_test)
predictions = predictions.to_series()

In [11]:
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

print('R2: ' + str(r2_score(y_test,  predictions)))
print('MSE: ' + str(mean_squared_error(y_test, predictions, squared = True)))
print('RMSE: ' + str(mean_squared_error(y_test, predictions, squared = False)))
print('MAE: ' + str(mean_absolute_error(y_test, predictions)))

R2: 0.9687633118367751
MSE: 3.7889379311728772
RMSE: 1.9465194402247508
MAE: 1.4187148945443582
